In [8]:
import glob
files = glob.glob("Fire images/*.*")
ls_fire = []
for i in files:
#     print([i.split("/")[1], '0'])
    ls_fire.append(['Fire images', i.split("/")[1], '1'])

In [9]:
import pandas as pd

df_fire = pd.DataFrame(ls_fire, columns=['Folder', 'filename', 'label'])

In [10]:
df_fire.head()

,Folder,filename,label


In [11]:
len(df_fire)

0

In [12]:
df_fire['filename'].duplicated().sum()

0

In [13]:
df_fire['filename'][0]

IndexError: index out of bounds

In [117]:
df_fire['Folder'][0]

'Fire images'

In [118]:
from PIL import Image
from keras.preprocessing.image import img_to_array

img = Image.open('{}/{}'.format(df_fire['Folder'][0], df_fire['filename'][0]))
print(img.size)
image_red = img.resize((1024, 1024))
image = img_to_array(image_red)

(3872, 2592)


In [119]:
image

array([[[ 70.,  25.,   2.],
        [ 69.,  27.,   3.],
        [ 67.,  26.,   0.],
        ...,
        [ 83.,  39.,  10.],
        [ 82.,  37.,   8.],
        [ 84.,  38.,   5.]],

       [[ 70.,  26.,   1.],
        [ 68.,  26.,   1.],
        [ 67.,  25.,   1.],
        ...,
        [ 82.,  38.,  13.],
        [ 77.,  32.,   0.],
        [ 82.,  37.,   4.]],

       [[ 70.,  26.,   1.],
        [ 71.,  29.,   4.],
        [ 70.,  29.,   1.],
        ...,
        [ 86.,  40.,  16.],
        [ 78.,  33.,   2.],
        [ 79.,  34.,   3.]],

       ...,

       [[ 24.,  16.,  13.],
        [ 26.,  18.,  15.],
        [ 28.,  20.,  17.],
        ...,
        [116.,  76.,  41.],
        [115.,  72.,  27.],
        [117.,  70.,  18.]],

       [[ 26.,  18.,  15.],
        [ 29.,  21.,  18.],
        [ 32.,  25.,  19.],
        ...,
        [101.,  66.,  36.],
        [ 92.,  55.,  10.],
        [ 95.,  57.,   8.]],

       [[ 32.,  24.,  21.],
        [ 28.,  20.,  17.],
        [ 32.,  

### Merge dfs

In [7]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import keras

#Split df into train and test based on label column
# df_train
# df_test
input_shape = (1024, 1024, 3)
num_classes = 1
batch_size = 16

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

def data_gen(df, batch_size):
    while True:
        x_batch = np.zeros((batch_size, 3, 1024, 1024))
        y_batch = np.zeros((batch_size, 1))
        for j in range(len(df/batch_size)):
            b = 0
            for m, k in zip(df['filename'].values[j*batch_size:(j+1)*batch_size], df['label'].values[j*batch_size:(j+1)*batch_size]):
                img = Image.open('{}/{}'.format(df['Folder'][b], m))
                image_red = img.resize((1024, 1024))
                x_batch[b] = img_to_array(image_red)
                y_batch[b] = k
                b += 1
            yield (x_batch, y_batch)


model.fit_generator(generator=data_gen(df_fire, batch_size=batch_size), steps_per_epoch=len(df_fire) // batch_size, epochs=epochs)

NameError: name 'df_fire' is not defined